TITLE OF YOUR MAIL :
    TDNLP ESILV 2023
    
PLEASE send me an email with your TD as
2023_{NOM}.html
(ex: my name is Maxence Jeunesse => 2023_JEUNESSE.html)
html OR pdf BUT NOT .iypnb NEITHER .py
to
maxence.jeunesse@covea.fr
DEADLINE : 17.11.2023

IMAGINE 
negativewords = ['WAR','CRISIS','DEFENSIVE']
positivewords = ['UP','LUCKY']

 = -1 -1 0 +1 = -1
computeMCLscore('WAR CRISIS IS LUCKY')
computeMCLscore('b"defensive, DEFENSIVE"') = -2

In [5]:
import numpy as np
import pandas as pd

Organisation :

    TD\
    - TD.ipynb
    - data\
          - Combined_News_DJIA.csv
          - upload_DJIA_table.csv
          - Loughran-McDonald_MasterDictionary_1993-2021.csv
          

requirements partie 1
- numpy
- pandas
- scikit-learn

requirements partie 2
- spacy


## Données utilisées : description

https://www.kaggle.com/datasets/aaron7sun/stocknews

I provided three data files in .csv format:

1. RedditNews.csv: two columns
The first column is the "date", and second column is the "news headlines".
All news are ranked from top to bottom based on how hot they are.
Hence, there are 25 lines for each date.

2. DJIA_table.csv:
Downloaded directly from Yahoo Finance: check out the web page for more info.

3. CombinedNewsDJIA.csv:
To make things easier for my students, I provide this combined dataset with 27 columns.
The first column is "Date", the second is "Label", and the following ones are news headlines ranging from "Top1" to "Top25".



To my students:

I made this a binary classification task. Hence, there are only two labels:

"1" when DJIA Adj Close value rose or stayed as the same;

"0" when DJIA Adj Close value decreased.

For task evaluation, please use data from 2008-08-08 to 2014-12-31 as Training Set, and Test Set is then the following two years data (from 2015-01-02 to 2016-07-01). This is roughly a 80%/20% split.

And, of course, use AUC as the evaluation metric.

In [6]:
%ls data\

 Le volume dans le lecteur D s'appelle Data
 Le num�ro de s�rie du volume est BCD0-88F8

 R�pertoire de d:\VSCodeWrokSpace\DATAA5\data

29/11/2023  09:10    <DIR>          .
29/11/2023  09:10    <DIR>          ..
29/11/2023  09:10         5�617�630 Combined_News_DJIA.csv
29/11/2023  09:10         8�880�507 Loughran-McDonald_MasterDictionary_1993-2021.csv
29/11/2023  09:10         9�099�659 RedditNews.csv
29/11/2023  09:10           167�083 upload_DJIA_table.csv
               4 fichier(s)       23�764�879 octets
               2 R�p(s)  760�524�320�768 octets libres


In [7]:
df = pd.read_csv(r'data\Combined_News_DJIA.csv')
df['Date']=pd.to_datetime(df['Date'])
djia = pd.read_csv(r'data\upload_DJIA_table.csv')
djia['Date'] = pd.to_datetime(djia['Date'])

# Approche 1 : Loughran McDonald

A partir du fichier Loughran-McDonald_MasterDictionary_1993-2021.csv pour l'année 2014

https://sraf.nd.edu/loughranmcdonald-master-dictionary/

1. calculer le sentiment pour chaque headline (25 au total) (+1 par mot "positif" et -1 par mot "negatif")
2. construisez une régression logistique qui prend en input le sentiment des 25 headlines et estime le signe de DJIA (- = 0, + = 1)
3. Calculez l'AUC sur la période test
4. Calculez le sharpe ratio sur la période test


## A propos du dictionnaire

The dictionary reports counts, proportion of total, average proportion per document, standard deviation of proportion per document, document count (i.e., number of documents containing at least one occurrence of the word), seven sentiment category identifiers,  number of syllables, and source for each word (source is either 12of12inf or the year in which the word was added).  
 
The sentiment categories are: negative, positive, uncertainty, litigious, strong modal, weak modal, and constraining. The sentiment words are flagged with a number indicating the year in which they were added to the list. Note: A year preceded by a negative sign indicates the year/version when the word was removed from the sentiment category.  
 
Although the dictionary does not, in general, include abbreviations, in the post-2018 versions we have added a limited number of abbreviations commonly occurring in the periodic filings

In [8]:
mcl=pd.read_csv(r'data/Loughran-McDonald_MasterDictionary_1993-2021.csv')

In [9]:
negativescope = mcl['Negative']>0
negativescope&= mcl['Negative']<2015

positivescope = mcl['Positive']>0
positivescope&= mcl['Positive']<2015


In [10]:
negative_words = set(mcl[negativescope]['Word'])
positive_words = set(mcl[positivescope]['Word'])

In [11]:
from collections import Counter

def computeMCLscore(txti):
    if pd.isnull(txti):
        return 0
    txt=txti[2:-1].upper()
    r = 0
    for word in txt.split():
        if word in positive_words:
            r += 1
        elif word in negative_words:
            r -= 1
    return r


In [12]:
labels = df[['Date','Label']].copy()
labels['Date']=pd.to_datetime(labels['Date'])
labels= labels.set_index('Date')
comp=djia.sort_values(by='Date').set_index('Date').diff().join(labels)
comp['LabeX']=(comp['Close']>0).astype('int')
(comp['Label']==comp['LabeX']).value_counts()

True    1989
Name: count, dtype: int64

In [13]:
from tqdm import tqdm

cols=['Top1', 'Top2', 'Top3', 'Top4', 'Top5', 'Top6', 'Top7',
       'Top8', 'Top9', 'Top10', 'Top11', 'Top12', 'Top13', 'Top14', 'Top15',
       'Top16', 'Top17', 'Top18', 'Top19', 'Top20', 'Top21', 'Top22', 'Top23',
       'Top24', 'Top25']
newdf=pd.DataFrame()
newdf['Date']=pd.to_datetime(df['Date'])
newdf['Label']=df['Label'].shift()
LMC_cols=[]
for c in tqdm(cols):
    newdf[c] = df[c]
    newdf['LMC'+c] = df[c].apply(computeMCLscore)
    LMC_cols.append('LMC'+c)
    
newdf = newdf.sort_values(by='Date',ascending=True)
newdf['Label']=newdf['Label'].shift(-1)
newdf=newdf.dropna()

 68%|██████▊   | 17/25 [00:00<00:00, 81.09it/s]

100%|██████████| 25/25 [00:00<00:00, 82.95it/s]


In [14]:
trainscope = newdf['Date']<pd.to_datetime('2015-01-01')
testscope = newdf['Date']>pd.to_datetime('2015-01-01')

In [15]:
X_train = newdf[trainscope][LMC_cols]
y_train = newdf[trainscope]['Label']

X_test = newdf[testscope][LMC_cols]
y_test = newdf[testscope]['Label']

In [16]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn import metrics

# Entraînement du modèle
logreg = LogisticRegressionCV(cv=5, penalty='l1', solver='liblinear', random_state=42)
logreg.fit(X_train, y_train)

# Prédiction sur l'ensemble de test
y_pred_prob = logreg.predict_proba(X_test)[:, 1]  # Probabilités pour la classe 1
y_pred = logreg.predict(X_test)  # Prédictions binaires


In [17]:
print(newdf)

           Date  Label                                               Top1   
0    2008-08-08    0.0  b"Georgia 'downs two Russian warplanes' as cou...  \
1    2008-08-11    1.0  b'Why wont America and Nato help us? If they w...   
2    2008-08-12    0.0  b'Remember that adorable 9-year-old who sang a...   
3    2008-08-13    0.0  b' U.S. refuses Israel weapons to attack Iran:...   
4    2008-08-14    1.0  b'All the experts admit that we should legalis...   
...         ...    ...                                                ...   
1983 2016-06-24    0.0  David Cameron to Resign as PM After EU Referendum   
1984 2016-06-27    0.0  Barclays and RBS shares suspended from trading...   
1985 2016-06-28    1.0  2,500 Scientists To Australia: If You Want To ...   
1986 2016-06-29    1.0                   Explosion At Airport In Istanbul   
1987 2016-06-30    1.0  Jamaica proposes marijuana dispensers for tour...   

      LMCTop1                                               Top2  LMCTop2  

In [18]:
computeMCLscore('b"Accidentally BENEFICIALLY BENEFICIALLY.')

1

In [19]:
from sklearn.metrics import roc_auc_score

auc_score = metrics.roc_auc_score(y_test, y_pred_prob)
print("AUC Score:", auc_score)



AUC Score: 0.5052356020942408


In [20]:
djia['Date'] = pd.to_datetime(djia['Date'])
djia = djia.sort_values(by='Date')
djia['Daily_Returns'] = djia['Close'].pct_change()

newdf_test = newdf[testscope]
newdf_test = newdf_test.merge(djia[['Date', 'Daily_Returns']], on='Date')


In [21]:
y_pred = logreg.predict(X_test)
predicted_returns = (y_pred * 2 - 1) * newdf_test['Daily_Returns']

mean_return = np.mean(predicted_returns)
std_return = np.std(predicted_returns)

sharpe_ratio = mean_return / std_return if std_return != 0 else 0
print("Sharpe Ratio:", sharpe_ratio)


Sharpe Ratio: -0.049726819119783904


In [22]:

sharpe_ratio = mean_return / std_return if std_return != 0 else 0
print("Sharpe Ratio:", sharpe_ratio)


Sharpe Ratio: -0.049726819119783904


# Approche 2 : LDA

On suppose qu'il y a des thèmes influencant le marché
On considère maintenant que toutes les news de la journée sont aggrégées.

1. Construire la matrice term document à partir des 200 termes les plus fréquents mais dont la fréquence d'apparition dans les documents est inférieure à 95% (i.e si le mot apparait dans 95% des documents, on le "jette")
2. A partir d'une LDA sur 10 topics, construire les poids des topics pour chaque journée
3. Utiliser ces poids dans une régression logistique
4. Calculer l'AUC et le sharpe ratio sur la période test

In [23]:
def row_aggregator(row):
    return '. '.join(x[2:-1] for x in row[cols] if pd.notnull(x))
newdf['txt_total']=newdf[cols].apply(row_aggregator,axis=1)

In [24]:
#!spacy download en_core_web_sm

In [25]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [26]:
#print(txt) doc.noun_chunks => chunk.text, chunk.root.text, chunk.root.head.text

In [27]:
def prep_txt_lda(txt):

    doc = nlp(txt)
    tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    txt = ' '.join(tokens)    
    return txt

In [28]:
try:
    from sklearn.decomposition import LatentDirichletAllocation as LDA
except:
    from sklearn.lda import LDA

In [29]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [32]:
from tqdm import tqdm
cnt = CountVectorizer(max_features=200, max_df=0.95)


clean_txt = [prep_txt_lda(txt) for txt in tqdm(newdf['txt_total'])]
newdf['clean_lda_txt']=clean_txt
dtm = cnt.fit_transform(newdf['clean_lda_txt'])

100%|██████████| 1985/1985 [03:08<00:00, 10.51it/s]


In [ ]:
from __future__ import print_function
import pyLDAvis
import pyLDAvis.lda_model
pyLDAvis.lda_model.prepare

pyLDAvis.enable_notebook()

In [73]:
# Préparation de la visualisation
#panel = pyLDAvis.prepare(lda, dtm, cnt, mds='tsne')
#panel
### marche pas

In [41]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components=10, random_state=0)
lda.fit(dtm)
topic_weights = lda.transform(dtm)


In [40]:
panel = pyLDAvis.lda_model.prepare(lda, dtm, cnt, mds='tsne')
pyLDAvis.display(panel)

In [45]:
print("Longueur de topic_weights:", len(topic_weights))
print("Longueur de djia['Daily_Returns']:", len(djia['Daily_Returns']))


Longueur de topic_weights: 1985
Longueur de djia['Daily_Returns']: 1989


In [59]:
print(len(topic_weights))
print(len(djia['Daily_Returns']))

1985
1989


In [60]:
# Enlever les quatre dernières valeurs
djia_reduced = djia.iloc[:-4]

# Vérifier la longueur après la suppression
print(len(djia_reduced['Daily_Returns']))


1985


In [62]:
# Vérifiez si des valeurs NaN sont présentes
print(djia_reduced['Daily_Returns'].isna().sum())


1


In [65]:
mean_value = djia_reduced['Daily_Returns'].mean()
djia_reduced['Daily_Returns'].fillna(mean_value, inplace=True)


C:\Users\paull\AppData\Local\Temp\ipykernel_11624\3547204034.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  djia_reduced['Daily_Returns'].fillna(mean_value, inplace=True)


In [76]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score


X_train_lda, X_test_lda, y_train_lda, y_test_lda = train_test_split(topic_weights, djia_reduced['Daily_Returns_Binary'], test_size=0.3, random_state=0)
lr_lda = LogisticRegression()
lr_lda.fit(X_train_lda, y_train_lda)

y_pred_prob_lda = lr_lda.predict_proba(X_test_lda)[:, 1]
y_pred_lda = lr_lda.predict(X_test_lda)

auc_score_lda = roc_auc_score(y_test_lda, y_pred_prob_lda)
accuracy_lda = metrics.accuracy_score(y_test_lda, y_pred_lda)

print("AUC Score (LDA):", auc_score_lda)
print("Précision (LDA):", accuracy_lda)


AUC Score (LDA): 0.4998304701627486
Précision (LDA): 0.5285234899328859


In [75]:
predicted_returns = (y_pred_lda * 2 - 1) * newdf_test['Daily_Returns']

mean_return = np.mean(predicted_returns)
std_return = np.std(predicted_returns)

sharpe_ratio = mean_return / std_return if std_return != 0 else 0
print("Sharpe Ratio:", sharpe_ratio)


ValueError: operands could not be broadcast together with shapes (596,) (377,) 